## Day 23: Amphipod

[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/mazharenko/AoC-2021/tree/HEAD/notebooks/day23/puzzle.ipynb)


In [ ]:
type Amphipod = | A | B | C | D
type Occupation = 
    | Free
    | Occupied of Amphipod
type Room = { Host : Amphipod; Occupation : Occupation list }
type Hall = { Occupation : Occupation; Entrance: Room option }
type Burrow = Hall[]

let cost = function | A -> 1 | B -> 10 | C -> 100 | D -> 1000
let hall occupation = { Occupation = occupation; Entrance = None }
let entrance room = { Occupation = Free; Entrance = Some room }

In [ ]:
let roomPop (room : Room) = 
    let (free, occupied) = List.partition ((=)Free) room.Occupation
    match occupied with 
    | (Occupied amphipod) :: roomRest ->
        amphipod,
        (List.length free + 1) * cost amphipod,
        {room with Occupation = Free :: free @ roomRest}
    | _ -> failwith "room is empty"

let roomPush (room : Room) amphipod = 
    let (free, occupied) = List.partition ((=)Free) room.Occupation
    match free with
    | [] -> failwith "room is full"
    | _::restFree -> 
        (free |> List.length) * cost amphipod, {room with Occupation = restFree @ [Occupied amphipod] @ occupied}

let hallPop (hall : Hall) = 
    match hall with
    | { Occupation = Free; Entrance = None } -> failwith "hall is free"
    | { Occupation = Free; Entrance = Some room } -> 
        let (amphipod, roomEnergySpent, newRoom) = roomPop room
        amphipod, roomEnergySpent, { hall with Entrance = Some newRoom }
    | { Occupation = Occupied amphipod } -> amphipod, 0, { hall with Occupation = Free }

let hallPush hall amphipod = 
    match hall.Entrance with
    | None -> 0, { hall with Occupation = Occupied amphipod }
    | Some room -> 
        let (energySpent, newRoom) = roomPush room amphipod
        energySpent, { hall with Entrance = Some newRoom }

In [ ]:

let samplePart1 = 
    [| 
        hall Free
        hall Free
        entrance { Host = A; Occupation = [Occupied B; Occupied A] }
        hall Free
        entrance { Host = B; Occupation = [Occupied C; Occupied D] }
        hall Free
        entrance { Host = C; Occupation = [Occupied B; Occupied C] }
        hall Free
        entrance { Host = D; Occupation = [Occupied D; Occupied A] }
        hall Free
        hall Free
    |]

let samplePart2 = 
    [|
        hall Free
        hall Free
        entrance { Host = A; Occupation = [Occupied B; Occupied D; Occupied D; Occupied A] }
        hall Free
        entrance { Host = B; Occupation = [Occupied C; Occupied C; Occupied B; Occupied D] }
        hall Free
        entrance { Host = C; Occupation = [Occupied B; Occupied B; Occupied A; Occupied C] }
        hall Free
        entrance { Host = D; Occupation = [Occupied D; Occupied A; Occupied C; Occupied A] }
        hall Free
        hall Free
    |]

In [ ]:
let actualPart1 = 
    [| 
        hall Free
        hall Free
        entrance { Host = A; Occupation = [Occupied D; Occupied B] }
        hall Free
        entrance { Host = B; Occupation = [Occupied C; Occupied A] }
        hall Free
        entrance { Host = C; Occupation = [Occupied D; Occupied A] }
        hall Free
        entrance { Host = D; Occupation = [Occupied B; Occupied C] }
        hall Free
        hall Free
    |]

let actualPart2 = 
    [| 
        hall Free
        hall Free
        entrance { Host = A; Occupation = [Occupied D; Occupied D; Occupied D; Occupied B] }
        hall Free
        entrance { Host = B; Occupation = [Occupied C; Occupied C; Occupied B; Occupied A] }
        hall Free
        entrance { Host = C; Occupation = [Occupied D; Occupied B; Occupied A; Occupied A] }
        hall Free
        entrance { Host = D; Occupation = [Occupied B; Occupied A; Occupied C; Occupied C] }
        hall Free
        hall Free
    |]

In [ ]:
let rec private simulate' (burrow : Burrow) acc =
    let rooms = burrow |> Array.choose (fun hall -> hall.Entrance)
    if rooms |> Array.forall (fun room -> room.Occupation |> List.forall ((=) (Occupied room.Host)))
    then [acc]
    else 
        // if any amphipod in the hall can go to its room it must be the best turn
        let amphipodToRoom = 
            (Array.indexed burrow, Array.indexed burrow)
            ||> Array.allPairs
            // amphipods in the hall and corresponding rooms
            |> Array.choose (fun ((iEntrance, hEntrance), (iAmph, hAmph)) -> 
                match hAmph.Occupation with 
                | Free -> None 
                | Occupied amph -> 
                    hEntrance.Entrance 
                    |> Option.filter (fun room -> room.Host = amph) 
                    |> Option.map (fun room -> iEntrance, room, iAmph, amph)
            )
            // room is empty or contains only matching amphipods
            |> Array.filter (fun (iEntrance, room, iAmph, amph) ->
                room.Occupation
                |> List.choose (function | Occupied amph -> Some amph | _ -> None)
                |> List.forall ((=) room.Host)
            )
            // no other amphipods on the way
            |> Array.filter (fun (iEntrance, room, iAmph, amph) ->
                if iEntrance < iAmph
                then burrow.[iEntrance..iAmph-1] 
                else burrow.[iAmph+1..iEntrance]
                |> Array.forall (fun h -> h.Occupation = Free)
            )
            |> Array.tryHead
        match amphipodToRoom with
        | Some (iEntrance, _, iAmph, amph) -> 
            let newBurrow = Array.copy burrow
            let (_, hallEnergySpent, newHallAmph) = hallPop burrow.[iAmph]
            let (roomEnergySpent, newEntrance) = hallPush burrow.[iEntrance] amph
            newBurrow.[iAmph] <- newHallAmph
            newBurrow.[iEntrance] <- newEntrance
            simulate' newBurrow acc
            |> List.map (fun x -> (roomEnergySpent + hallEnergySpent + (cost amph) * (abs (iEntrance - iAmph))) + x)
        | None -> 
            // try taking amphipods from rooms 
            let amphipodFromRoom = 
                (Array.indexed burrow, Array.indexed burrow)
                ||> Array.allPairs
                // room and free hall
                |> Array.choose (fun ((iEntrance, hEntrance), (iFree, hFree)) -> 
                    match hEntrance, hFree with
                    | { Entrance = Some room }, {Occupation = Free; Entrance = None} -> 
                        Some (iEntrance, room, iFree)
                    | _ -> None
                )
                // room contains an unmatching amphipod
                |> Array.filter (fun (iEntrance, room, iFree) -> 
                    room.Occupation 
                    |> List.filter ((<>) (Free)) 
                    |> List.filter ((<>) (Occupied room.Host))
                    |> List.length > 0
                )
                // no other amphipods on the way
                |> Array.filter (fun (iEntrance, room, iFree) -> 
                    if iEntrance < iFree
                    then burrow.[iEntrance..iFree-1] 
                    else burrow.[iFree+1..iEntrance]
                    |> Array.forall (fun h -> h.Occupation = Free)
                )
                |> List.ofSeq
            amphipodFromRoom 
            |> List.collect (fun (iEntrance, room, iFree) -> 
                let newBurrow = Array.copy burrow
                let (amphipod, entranceEnergySpent, newEntrance) = hallPop burrow.[iEntrance]
                let (hallEnergySpent, newHall) = hallPush burrow.[iFree] amphipod

                newBurrow.[iFree] <- newHall
                newBurrow.[iEntrance] <- newEntrance

                simulate' newBurrow acc
                |> List.map (fun x -> (entranceEnergySpent + hallEnergySpent + (cost amphipod) * (abs (iEntrance - iFree))) + x)
            )
let simulate (burrow : Burrow) = simulate' burrow 0 |> List.min



In [ ]:
simulate samplePart1

12521

In [ ]:
simulate actualPart1

14371

In [ ]:
simulate samplePart2

44169

In [ ]:
simulate actualPart2

40941